## Solving Bateman-Burgers Equation with K-T Scheme

In [7]:
from numpy import pi, sin, abs, min, max
import matplotlib.pyplot as plt
from matplotlib import animation

from kurganov_tadmor import *

### The Burgers equation

We solve the inviscid Burgers' equation on the domain
$$ \partial_t u + \partial_x \left( \frac{u^2}{2}\right) = 0, $$

on the domain $(x,t)\in([0,2\pi]\times[0,10])$ with initial data
$$ u(x,0) = \sin x + \frac{1}{2} \sin\left(\frac{x}{2}\right)\,. $$

and periodic boundary conditions. The solution is computed using 400 cells and CFL number is taken as 0.75 (Time-marching).


In [23]:
pars = Pars1d(
    x_init=0.0,
    x_final=2.0 * pi,
    t_final=10,
    dt_out=0.1,
    J=400,
    cfl=0.75,
    scheme="sd3")

In [24]:
class Burgers1d(Equation1d):
    def initial_data(self):
        x = self.x
        return sin(x) + 0.5 * sin(0.5 * x)

    def boundary_conditions(self, u):
        u[0] = u[-4]
        u[1] = u[-3]
        u[-2] = u[2]
        u[-1] = u[3]

    def flux_x(self, u):
        return 0.5 * u**2

    def spectral_radius_x(self, u):
        return abs(u)

### Solve the equation

In [25]:
eqn = Burgers1d(pars)
soln = Solver1d(eqn)
soln.solve()

In [28]:
soln.__dict__.keys()

dict_keys(['x_init', 'x_final', 't_final', 'dt_out', 'J', 'cfl', 'scheme', 'Nt', 'x', 'dx', 'odd', 'dt', 'step', 'flux_x', 'boundary_conditions', 'spectral_radius_x', 'u', 'u_n'])

### Animate the solution

In [22]:
j0 = slice(2, -2)

fig = plt.figure()
ax = plt.axes(xlim=(soln.x_init,soln.x_final), ylim=(min(soln.u_n), max(soln.u_n)*1.1))

line_u, = ax.plot([], [], linewidth=1, marker='o', markersize=2)
ax.set_xlabel('x')
ax.set_ylabel('u')
plt.legend(["burgers"], loc=3, frameon=False)

def init():
    line_u.set_data([], [])

def animate(i):
    line_u.set_data(soln.x[j0], soln.u_n[i,j0])
    
plt.close()
anim = animation.FuncAnimation(fig, animate, init_func=init, frames=soln.Nt, interval=100, blit=False);
anim.save('results/burgers.mp4')